In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

In [3]:
# Load a dataset - Example: IMDb for sentiment analysis
dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
# Load BERT tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Load RoBERTa tokenizer and model
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Tokenization function for BERT
def tokenize_for_bert(examples):
    return bert_tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenization function for RoBERTa
def tokenize_for_roberta(examples):
    return roberta_tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenize dataset for BERT
tokenized_datasets_bert = dataset.map(tokenize_for_bert, batched=True)

# Tokenize dataset for RoBERTa
tokenized_datasets_roberta = dataset.map(tokenize_for_roberta, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
# Define training arguments for BERT
training_args_bert = TrainingArguments(
    output_dir='./bert-results',      # Output directory
    learning_rate=2e-5,               # Learning rate
    per_device_train_batch_size=8,    # Batch size for training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    num_train_epochs=3,               # Number of epochs
    weight_decay=0.01,                # Weight decay
    evaluation_strategy="epoch",      # Evaluate at the end of every epoch
    fp16=True,                        # Enable mixed precision for GPU training
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
# Define training arguments for RoBERTa
training_args_roberta = TrainingArguments(
    output_dir='./roberta-results',   # Output directory
    learning_rate=2e-5,               # Learning rate
    per_device_train_batch_size=8,    # Batch size for training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    num_train_epochs=3,               # Number of epochs
    weight_decay=0.01,                # Weight decay
    evaluation_strategy="epoch",      # Evaluate at the end of every epoch
    fp16=True,                        # Enable mixed precision for GPU training
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
# Trainer for BERT
trainer_bert = Trainer(
    model=bert_model,
    args=training_args_bert,
    train_dataset=tokenized_datasets_bert['train'],
    eval_dataset=tokenized_datasets_bert['test'],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [9]:
# Trainer for RoBERTa
trainer_roberta = Trainer(
    model=roberta_model,
    args=training_args_roberta,
    train_dataset=tokenized_datasets_roberta['train'],
    eval_dataset=tokenized_datasets_roberta['test'],
)


In [10]:
# Fine-tune BERT
trainer_bert.train()

Epoch,Training Loss,Validation Loss
1,0.254300,0.261553
2,0.146100,0.267922
3,0.071000,0.311927


TrainOutput(global_step=9375, training_loss=0.17203111104329427, metrics={'train_runtime': 3061.6148, 'train_samples_per_second': 24.497, 'train_steps_per_second': 3.062, 'total_flos': 1.9733329152e+16, 'train_loss': 0.17203111104329427, 'epoch': 3.0})

In [11]:
# Fine-tune RoBERTa
trainer_roberta.train()

Epoch,Training Loss,Validation Loss
1,0.240800,0.213609
2,0.143200,0.232935
3,0.084700,0.267710


TrainOutput(global_step=9375, training_loss=0.18112766764322916, metrics={'train_runtime': 4539.7608, 'train_samples_per_second': 16.521, 'train_steps_per_second': 2.065, 'total_flos': 1.9733329152e+16, 'train_loss': 0.18112766764322916, 'epoch': 3.0})

In [12]:
# Evaluate BERT
bert_eval_results = trainer_bert.evaluate()

# Evaluate RoBERTa
roberta_eval_results = trainer_roberta.evaluate()

# Print evaluation results
print("BERT Evaluation Results:", bert_eval_results)
print("RoBERTa Evaluation Results:", roberta_eval_results)


BERT Evaluation Results: {'eval_loss': 0.31192681193351746, 'eval_runtime': 201.6945, 'eval_samples_per_second': 123.95, 'eval_steps_per_second': 7.749, 'epoch': 3.0}
RoBERTa Evaluation Results: {'eval_loss': 0.26771029829978943, 'eval_runtime': 298.7133, 'eval_samples_per_second': 83.692, 'eval_steps_per_second': 5.232, 'epoch': 3.0}
